<a href="https://colab.research.google.com/github/pakka79/pakka79/blob/main/SellPoint_Volume%2BATR%2BVWAP%2BBollingerBand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import talib as ta
import pandas_datareader as pdr
import csv

from pykrx import stock
from pykrx import bond
import yfinance as yf
import datetime as dt
import pytz

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

plt.style.use("fivethirtyeight")
yf.pdr_override() 

# #Build Class

In [ ]:
class KRX_Input(object):

    def __init__(self,Today,Previousday,Ticker):
        self.Today=Today
        self.Previousday=Previousday
        self.Ticker=Ticker  
        
    def print_info(self):
        print("Ticker",self.Ticker)
        print("Today",self.Today)
        print("Previousday",self.Previousday)

    def KRX_Output(self):
        df = stock.get_market_ohlcv(self.Previousday,self.Today,self.Ticker , "d")
        df.columns = ['Open','High','Low','Close','Volume']
        for Ticker in stock.get_market_ticker_list()[0]:
            Corp = stock.get_market_ticker_name(self.Ticker) 

        #Build ATR

        Mark=[]
        W_ATR=2.5

        df['ATR'] = ta.ATR(df['High'],df['Low'],df['Close'], timeperiod=20)
        df['W_ATR']=df['ATR']*W_ATR
        df['Range'] = df['High']-df['Low']


        for i in range(-20,len(df)):
            if df.ATR.iloc[i]*W_ATR>df.Range.iloc[i] :
                Mark.append("")
            elif df.ATR.iloc[i]*W_ATR<df.Range.iloc[i] :
                Mark.append("v")

        df['Mark']=Mark

        Mark2=[]
        for i in range(-20,len(df)):
            if df.ATR.iloc[i]*W_ATR>df.Range.iloc[i] :
                Mark2.append("")
            elif df.ATR.iloc[i]*W_ATR<df.Range.iloc[i] :
                Mark2.append(df['Close'][i])
        df['Mark2']=Mark2

        #Build Bollinger Band
        def Build_Bollinger_Band(df):
            df['SMA_20']=df.Close.rolling(20).mean()
            df['SMA_5']=df.Close.rolling(5).mean()
            df['stddev']=df.Close.rolling(5).std()
            df['Upper']=df.SMA_5 + 2.5*df.stddev
            df['Lower']=df.SMA_5 - 2.5*df.stddev
            df['rsi']=ta.RSI(df['Close'],timeperiod=14)
        Build_Bollinger_Band(df)

        #Build OBV
        OBV = []
        OBV.append(0)
        for i in range(1, len(df.Close)):
            if df.Close[i] > df.Close[i-1]: #If the closing price is above the prior close price 
                OBV.append(OBV[-1] + df.Volume[i]) #then: Current OBV = Previous OBV + Current Volume
            elif df.Close[i] < df.Close[i-1]:
                OBV.append( OBV[-1] - df.Volume[i])
            else:
                OBV.append(OBV[-1])
        
        #Store the OBV and OBV EMA into new columns
        df['OBV'] = OBV
        df['OBV_EMA'] = df['OBV'].ewm(com=20).mean()

        #Build MACD
        def MACD(df):
            df['EMA12'] = df.Close.ewm(span=12,).mean()
            df['EMA26'] = df.Close.ewm(span=26,).mean()
            df['MACD'] = df.EMA12-df.EMA26
            df['signal'] = df.MACD.ewm(span=9).mean()           
        MACD(df)

        #Build VWAP
        v = df['Volume'].values
        tp = (df['Low'] + df['Close'] + df['High']).div(3).values
        df = df.assign(vwap=(tp * v).cumsum() / v.cumsum())

        #Ploting
        StockPrice ="_Stock_Price (Won)"
        fig = make_subplots(specs=[[{"secondary_y": True}]]) #<===== 동알한 Graph tpye에서만 실행 가능#

        #Line Input#
        fig.add_trace(go.Scatter(x=df.index,y=df['vwap'],mode='lines',name='vwap', line=dict(color='gray',width=2)), secondary_y=False,)

        fig.add_trace(go.Scatter(x=df.index,y=df["SMA_5"],mode='lines',name='SMA_5', line=dict(color='yellow',width=2)), secondary_y=False,)
        fig.add_trace(go.Scatter(x=df.index,y=df["Upper"],mode='lines',name='Upper', line=dict(color='green',width=2)), secondary_y=False,)
        fig.add_trace(go.Scatter(x=df.index,y=df["Lower"],mode='lines',name='Lower', line=dict(color='red',width=2)), secondary_y=False,)

        fig.add_trace(go.Scatter(x=df.index,y=df["MACD"],mode='lines',name='MACD', line=dict(color='orange',width=2)), secondary_y=False,)
        fig.add_trace(go.Scatter(x=df.index,y=df["signal"],mode='lines',name='signal', line=dict(color='purple',width=2)), secondary_y=False,)

        fig.add_trace(go.Scatter(x=df.index,y=df["Close"],mode='lines',name='Close', line=dict(color='black',width=2)), secondary_y=False,)

        fig.add_trace(go.Scatter(x=df.index,y=df["W_ATR"],mode='lines',name='W_ATR', line=dict(color='gray',width=3)), secondary_y=True,)

        #Scattter Input#
        fig.add_trace(go.Scatter(x=df.index,y=df['Mark2'], mode='markers',marker=dict(size=9, color='blue', symbol=6),marker_size=15,name="W_ATR"+str(W_ATR)+"배 이상"), secondary_y=False,)

        #Fig Input#
        fig.update_layout(height=600,width=1800,title=Corp+Today,yaxis_title=Corp + StockPrice)
        fig.update_layout(xaxis_rangeslider_visible=False)

        #Fig2 Input#

        fig2 = go.Figure((go.Bar(x=df.index,y=df['Volume'],name='Volume')))

        #Line Input2#

        fig2.add_trace(go.Scatter(x=df.index,y=df['OBV_EMA'],mode='lines',name='OBV_EMA'+Corp, line=dict(color='purple',width=2)))
        fig2.add_trace(go.Scatter(x=df.index,y=df['OBV'],mode='lines',name='OBV'+Corp, line=dict(color='orange',width=2)))

        #Fig Input2#
        fig2.update_layout(height=600,width=1800,title=Corp+Today,yaxis_title=Corp + StockPrice)
        fig2.update_layout(xaxis_rangeslider_visible=False)

        fig.show()
        fig2.show()



## Input 

In [ ]:
Ticker="005930"
KoreaTime = pytz.timezone('Asia/Seoul')
From_Today=2000
today = dt.datetime.now()
previouday=today-dt.timedelta(days=From_Today)
Today=today.strftime("%Y%m%d")
Previuosday=previouday.strftime("%Y%m%d")

input=KRX_Input(Today,Previuosday,Ticker)
input.KRX_Output()